In [22]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime
import gc

In [23]:
files = ['CW001292_hist.xls','CW001292_TOU.xls']
meter_no = files[0].split("_")[0].strip()
meter_no

'CW001292'

In [24]:
df = pd.read_table(files[0])
#df

In [45]:
#df.columns[0].split(":")[0]
df = pd.read_table(files[0])
old_name = df.columns[0]
new_name = df.columns[0].split(":")[0]

In [ ]:
x = str(df.iloc[0]).split(":")[1].strip().split(" ")[0]
x.split(":")[1].strip().split(" ")[0]

'2025-02-01'

In [64]:
bill_date = str(df.iloc[0]).split(":")[1].strip().split(" ")[0]
billing_date = datetime.strptime(bill_date, '%Y-%m-%d').date()
billing_date

datetime.date(2025, 2, 1)

In [62]:
date_str = '2025-02-01'
date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
date_obj

datetime.date(2025, 2, 1)

In [ ]:
df = df.reset_index().rename(columns={'index': 'parameters',df.columns[0]:'energy_values'})
df.dropna(inplace=True)
#df.rename(columns={df.columns[0]: 'new_name'}, inplace=True)

In [27]:
df.dropna(inplace=True)
#df = df.head(2)
df

,parameters,energy_values
1,Active Energy Imp. (MWh),20288.5
2,Active Energy Exp. (MWh),98499.9
3,Reactive Energy QI (Mvarh),390.4
4,Reactive Energy QII (Mvarh),35733.2
5,Reactive Energy QIII (Mvarh),3588.0
6,Reactive Energy QIV (Mvarh),4226.1
7,Apparent Energy Imp. (MVAh),20908.4
8,Apparent Energy Exp. (MVAh),110979.1
9,Reactive Energy Imp. (Mvarh),36123.6
10,Reactive Energy Exp. (Mvarh),7814.1


In [28]:
df['parameters'] = df['parameters'].str.replace(" ","_").str.lower().str.strip().str.split(".", expand=True)[0]
df

,parameters,energy_values
1,active_energy_imp,20288.5
2,active_energy_exp,98499.9
3,reactive_energy_qi_(mvarh),390.4
4,reactive_energy_qii_(mvarh),35733.2
5,reactive_energy_qiii_(mvarh),3588.0
6,reactive_energy_qiv_(mvarh),4226.1
7,apparent_energy_imp,20908.4
8,apparent_energy_exp,110979.1
9,reactive_energy_imp,36123.6
10,reactive_energy_exp,7814.1


In [29]:
values_needed = ['active_energy_imp','active_energy_exp','apparent_energy_imp',
                 'reactive_energy_imp','reactive_energy_exp']
df = df[df['parameters'].isin(values_needed)]
df

,parameters,energy_values
1,active_energy_imp,20288.5
2,active_energy_exp,98499.9
7,apparent_energy_imp,20908.4
9,reactive_energy_imp,36123.6
10,reactive_energy_exp,7814.1


In [30]:
df_rates = pd.read_table(files[1])
df_rates = df_rates.rename(columns={df_rates.columns[0]: 'parameters'})
units = df_rates.loc[0, 'parameters'].split(".")[1].strip().strip("()")
if units == 'MWh':
    units_multiplier = 1000

elif units == 'kWh':
    units_multiplier = 1

elif units == 'GWh':
    units_multiplier = 1000000

units_multiplier

1000

In [31]:
df_rates['parameters'] = df_rates['parameters'].str.replace(" ","_").str.lower().str.strip().str.split(".", expand=True)[0]
df_rates = df_rates.head(2)
df_rates

,parameters,Rate 1,Rate 2,Rate 3,Rate 4,Rate 5,Rate 6,Rate 7,Rate 8
0,active_energy_imp,11406.8,4289.1,4592.6,0.0,0.0,0.0,0.0,0.0
1,active_energy_exp,50296.7,30065.7,18615.6,0.0,0.0,0.0,0.0,0.0


In [32]:
#df_final = df.merge(df_rates, how='inner', on="parameters")

In [33]:
cumulative_import = df.loc[df['parameters'] == 'active_energy_imp', 'energy_values'].values[0]
cumulative_import

'20288.5'

In [34]:
cumulative_import = df.loc[df['parameters'] == 'active_energy_imp', 'energy_values'].values[0]
cumulative_export = df.loc[df['parameters'] == 'active_energy_exp', 'energy_values'].values[0]
apparent_import = df.loc[df['parameters'] == 'apparent_energy_imp', 'energy_values'].values[0]
reactive_import = df.loc[df['parameters'] == 'reactive_energy_imp', 'energy_values'].values[0]
reactive_export = df.loc[df['parameters'] == 'reactive_energy_exp', 'energy_values'].values[0]

rate1 = df_rates.loc[0, 'Rate 1']
rate2 = df_rates.loc[0, 'Rate 2']
rate3 = df_rates.loc[0, 'Rate 3']

rate4 = df_rates.loc[1, 'Rate 1']
rate5 = df_rates.loc[1, 'Rate 2']
rate6 = df_rates.loc[1, 'Rate 3']

In [43]:
billing_values = {'cumulative_import': cumulative_import,'cumulative_export': cumulative_export,'apparent_import': apparent_import,
    'reactive_import': reactive_import, 'reactive_export': reactive_export, 'rate1': rate1, 'rate2': rate2, 'rate3': rate3,
    'rate4': rate4,'rate5': rate5,'rate6': rate6
}
for key, value in billing_values.items():
    billing_values[key] = float(value) * units_multiplier

billing_values

{'cumulative_import': 20288500.0,
 'cumulative_export': 98499900.0,
 'apparent_import': 20908400.0,
 'reactive_import': 36123600.0,
 'reactive_export': 7814100.0,
 'rate1': 11406800.0,
 'rate2': 4289100.0,
 'rate3': 4592600.0,
 'rate4': 50296700.0,
 'rate5': 30065700.0,
 'rate6': 18615600.0}

In [44]:

columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset', 'programing_count',
    'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime']

for key in columns_to_assign_na:
    billing_values[key] = pd.NA
billing_values

{'cumulative_import': 20288500.0,
 'cumulative_export': 98499900.0,
 'apparent_import': 20908400.0,
 'reactive_import': 36123600.0,
 'reactive_export': 7814100.0,
 'rate1': 11406800.0,
 'rate2': 4289100.0,
 'rate3': 4592600.0,
 'rate4': 50296700.0,
 'rate5': 30065700.0,
 'rate6': 18615600.0,
 'vt_ratio': <NA>,
 'ct_ratio': <NA>,
 'no_of_resets': <NA>,
 'date_of_last_reset': <NA>,
 'programing_count': <NA>,
 'max_dem1': <NA>,
 'max_dem1_datetime': <NA>,
 'max_dem2': <NA>,
 'max_dem2_datetime': <NA>,
 'max_dem3': <NA>,
 'max_dem3_datetime': <NA>}

In [36]:
new_df = pd.DataFrame(columns=['cumulative_import', 'cumulative_export','apparent_import','reactive_import',
    'reactive_export','rate1', 'rate2', 'rate3', 'rate4', 'rate5', 'rate6'])

row1 = pd.DataFrame({
    'cumulative_import': [cumulative_import],
    'cumulative_export': [cumulative_export],
    'apparent_import': [apparent_import],
    'reactive_import': [reactive_import],
    'reactive_export': [reactive_export],
    'rate1': [rate1],
    'rate2': [rate2],
    'rate3': [rate3],
    'rate4': [rate4],
    'rate5': [rate5],
    'rate6': [rate6]
})

new_df = pd.concat([new_df, row1], ignore_index = True)
new_df

C:\Users\HP\AppData\Local\Temp\ipykernel_27116\1270053271.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, row1], ignore_index = True)


,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,rate1,rate2,rate3,rate4,rate5,rate6
0,20288.5,98499.9,20908.4,36123.6,7814.1,11406.8,4289.1,4592.6,50296.7,30065.7,18615.6


In [37]:
columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset', 'programing_count',
    'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime']

for column in columns_to_assign_na:
    new_df[column] = pd.NA

new_df

,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,rate1,rate2,rate3,rate4,rate5,...,ct_ratio,no_of_resets,date_of_last_reset,programing_count,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime
0,20288.5,98499.9,20908.4,36123.6,7814.1,11406.8,4289.1,4592.6,50296.7,30065.7,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [38]:
new_df.columns

Index(['cumulative_import', 'cumulative_export', 'apparent_import',
       'reactive_import', 'reactive_export', 'rate1', 'rate2', 'rate3',
       'rate4', 'rate5', 'rate6', 'vt_ratio', 'ct_ratio', 'no_of_resets',
       'date_of_last_reset', 'programing_count', 'max_dem1',
       'max_dem1_datetime', 'max_dem2', 'max_dem2_datetime', 'max_dem3',
       'max_dem3_datetime'],
      dtype='object')

In [39]:
new_df['apparent_import']

0    20908.4
Name: apparent_import, dtype: object